[![img/pythonista.png](img/pythonista.png)](https://www.pythonista.io)

# Introducción a *Dataframes*.

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Intro a Dataframes").getOrCreate()
ct = spark.sparkContext

Los *dataframes* son un concepto compartido entre plataformas como *R*, *Pandas* y *Scala*. Son estructuras tabulares en las que todos los datos de una columna comparten el mismo tipado. Cada columna tiene un título y cada renglón tiene un índice. A la descripción de los tipos de datos de cada columna de un *dataframe* se le conoce como esquema (*schema*).

*PySPark* tiene la capacidad de poder manejar *dataframes* tanto de *Pandas* como de *Spark* e incluso cuenta con una [*API*](https://spark.apache.org/docs/latest/api/python/user_guide/pandas_on_spark/index.html) que optimiza la interacción entre ambos tipos de *dataframes*.

## Los *dataframes* de *Spark*.

Un *dataframe* de *Spark* es un objeto instanciado de la clase [```pyspark.sql.DataFrame```](https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.sql.DataFrame.html).

### Creación de *dataframes*.

La función [```spark.createDataFrame()```](https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.sql.SparkSession.createDataFrame.html) permite crear *dataframes* a partir de objetos que se ingresan como argumentos.


```pyspark
df = spark.createDataFrame(data=<obj>, <títulos columnas>, schema=<esquema>)
```

Donde:

* ```<obj>``` es un objeto que represente una estructura tabular el cual puede ser:
    * Una colección de *Python*.
    * Un *dataframe* de *Pandas*.
    * Un *RDD* de *Spark*.
* ```<titulos columnas>``` es una lista de cadenas de caracteres que corresponden al título de cada columna.
* ```<esquema>``` es una estructura de tipos de datos de *Spark* que describe el tipado de cada columna.
    
Cabe hacer notar que los *dataframes* de *Spark* se construyen de forma perezosa, por lo que aún cuando sean definidos, estos no serán creados hasta que sean requeridos.
    
Por convención se utiliza el nombre ```df``` para designar un dataframe genérico.

### Los objetos ```Row``` y ```Column```.

Los *dataframes* de *Spark* están compuestos por 2 tipos de objetos.

* Los objetos de tipo ```pyspark.sql.Row``` que representan a cada renglón del *dataframe*.
* Los objetos de tipo ```pyspark.sql.Column``` que representan a cada columna del *dataframe*.

## Diferencias entre *RDD* y *Dataframes*.

Los *RDD* de *Apache Spark*. Son colecciones de datos que pueden ser crudos (incluyendo binarios), semiestructurados (*JSON*, *XML*, *YAML*) o estructurados. Al final de cuentas los *RDD* funcionan de forma similar a un *data lake*, mientras que los *dataframes* siempre tiene una estructura de tabla.

Los *RDD* son creados desde el contexto de *Spark*, mientras que los dataframes son creados usando la *API* de *SQL*.
https://spark.apache.org/docs/2.4.0/api/python/pyspark.sql.html

### El método ```df.show()```.

El método ```df.show()``` muestra los primeros ```n``` números de un *dataframe* de *Spark*.

```
df.show(<n>)
```

Donde:

* ```<n>``` es el número de renglones desplegados. El valor por defecto es ```20```.

**Ejemplos:**

* Las siguientes celdas crearán un *dataframe* de *Spark* a partir de un *dataframe* de *Pandas*.

In [2]:
import pandas as pd

* Se creará el *dataframe* de *Pandas* llamado ```pandas_df```.

In [3]:
pandas_df = pd.DataFrame({'Dirección':('Sur', 'Norte', 'Sur', 'Este'),
              'Rumbo':('Este', 'Noroeste', 'Norte', 'Norte'),
             'Pasajeros':(12, 24, 32, 5),
             'Documentado':(True, None, True, False) })

In [4]:
pandas_df

,Dirección,Rumbo,Pasajeros,Documentado
0,Sur,Este,12,True
1,Norte,Noroeste,24,None
2,Sur,Norte,32,True
3,Este,Norte,5,False


In [5]:
pandas_df.dtypes

Dirección      object
Rumbo          object
Pasajeros       int64
Documentado    object
dtype: object

* Se creará el *dataframe* de *Spark* a partir de ```pandas_df```.

In [6]:
df = spark.createDataFrame(pandas_df)

/usr/local/spark/python/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/usr/local/spark/python/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [7]:
df.show()

+---------+--------+---------+-----------+
|Dirección|   Rumbo|Pasajeros|Documentado|
+---------+--------+---------+-----------+
|      Sur|    Este|       12|       true|
|    Norte|Noroeste|       24|       null|
|      Sur|   Norte|       32|       true|
|     Este|   Norte|        5|      false|
+---------+--------+---------+-----------+



In [8]:
df.show(2)

+---------+--------+---------+-----------+
|Dirección|   Rumbo|Pasajeros|Documentado|
+---------+--------+---------+-----------+
|      Sur|    Este|       12|       true|
|    Norte|Noroeste|       24|       null|
+---------+--------+---------+-----------+
only showing top 2 rows



In [9]:
df.schema

StructType([StructField('Dirección', StringType(), True), StructField('Rumbo', StringType(), True), StructField('Pasajeros', LongType(), True), StructField('Documentado', BooleanType(), True)])

* Las siguientes celdas creará un *dataframe* de *Spark* a partir dde un *RDD* que contiene una colección de objetos ```Row```.

In [10]:
from pyspark.sql import Row

* La siguiente celda creará un *RDD* de *Spark* que contiene una sucesión de objetos tipo ```Row```.
    * El objeto ```rdd``` tiene una estructura que puede ser convertida en una tabla.

In [11]:
rdd = ct.parallelize((Row('Sur', 'Este', '12', True),
                     Row('Norte', 'Noroeste', '24', None),
                     Row('Sur', 'Norte', '32', True),
                     Row('Este', 'Norte', '5', False)))

In [12]:
rdd_2 = ct.parallelize((Row('Sur', 'Este', '12', True),
                     1, 2, 3, 4,
                     Row('Sur', 'Norte', '32', True),
                     Row('Este', 'Norte', '5', False),
                      'uno', 'dos', b'121312412413523513462156725624567'))

* La siguiente celda creará un *dataset* a partir del *RDD* y además se ingresará como argumento el nombre de cada columna.

In [13]:
df = spark.createDataFrame(rdd, 
                ['Dirección', 'Rumbo', 'Pasajeros', 'Documentado'])

In [14]:
df.show()

+---------+--------+---------+-----------+
|Dirección|   Rumbo|Pasajeros|Documentado|
+---------+--------+---------+-----------+
|      Sur|    Este|       12|       true|
|    Norte|Noroeste|       24|       null|
|      Sur|   Norte|       32|       true|
|     Este|   Norte|        5|      false|
+---------+--------+---------+-----------+



* La siguiente celda creará un *dataframe* al que se le asignarán los nombres de las columnas automáticamente.

In [15]:
df_1 = spark.createDataFrame(rdd)
df_1.show()

+-----+--------+---+-----+
|   _1|      _2| _3|   _4|
+-----+--------+---+-----+
|  Sur|    Este| 12| true|
|Norte|Noroeste| 24| null|
|  Sur|   Norte| 32| true|
| Este|   Norte|  5|false|
+-----+--------+---+-----+



### Selección de una columna de un *dataframe*.

Es posible acceder a una columna de un *dataframe* usando su nombre o usando su número de índice consecutivo iniciando desde ```0```. 

```
df.<Nombre Columna>
```

```
df[<n>]
```


In [16]:
df.Rumbo

Column<'Rumbo'>

In [17]:
df[1]

Column<'Rumbo'>

### El atributo ```df.schema```.

El atributo ```df.schema``` contiene la estructura del *schema* del *dataframe* como una instancia de la clase ```pyspark.sql.types.StructType``` que contiene una colección de instancias de tipo ```pyspark.sql.types.StructField```.

**Ejemplo:**

In [18]:
df.schema

StructType([StructField('Dirección', StringType(), True), StructField('Rumbo', StringType(), True), StructField('Pasajeros', StringType(), True), StructField('Documentado', BooleanType(), True)])

### El método ```df.printSchema()```.

El método ```df.printSchema()``` despliega una cadena de caractéres describiendo el *schema* del *dataframe*.




In [19]:
df.printSchema()

root
 |-- Dirección: string (nullable = true)
 |-- Rumbo: string (nullable = true)
 |-- Pasajeros: string (nullable = true)
 |-- Documentado: boolean (nullable = true)



## Tipado de *Spark*.

*Spark* cuenta con distintos tipos de datos que extienden a los tipos nativos de *Scala*, *Python* y *R*. El módulo ```pyspark.sql.types``` contiene a todas las clases correspondientes a dichos tipos.

El siguente enlace apunta a la referncia de tipos de datos de *Spark*.

https://spark.apache.org/docs/latest/sql-ref-datatypes.html

* La siguiente celda importará todos los tipos de datos de *PySpark* al entorno de esta *notebook*.

In [20]:
from pyspark.sql.types import *

## Definición de *schemas* .

Los *schemas* describen estructuras de tipos de datos en *Spark*. Estas estructuras no sólo describen tablas.

### El tipo ```pyspark.api.types.StructField```.

El tipo [```pyspark.api.types.StructField```](https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.types.StructField.html) permite definir esquemas tanto para describir las columnas de un *dataframe* como para describir *schemas* complejos como los que se pueden encontrar en *YAML* o *JSON*.

```
StructField(<identificador>, <tipo>, <nulificable>)
```
Donde:

* ```<identificador>``` es el nombre del campo.
* ```<tipo>``` es el tipo de dato del campo.
* ```<nulificable>``` es un valor booleano que indica si el campo puede aceptar valores nulos.


Las estructuras complejas que pueden crearse en los *schemas* son contenidas dentro de un objeto de tipo [```pyspark.api.types.StructType```](https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.types.StructType.html).

```
StructType(<campo 1>, <campo 2>, ... , <campo n>)
```

Donde:

* ```<campo i>``` es un objeto de tipo ```StructField```.

**Ejemplo:**

* La siguiente celda describe un *schema* para el *dataframe* ```df``` en el que la columna ```Pasajeros``` es un entero que va de ```-128```a ```127```.

In [21]:
schema = StructType([StructField('Dirección', StringType(), True), 
                     StructField('Rumbo', StringType(), True), 
                     StructField('Pasajeros', ByteType(), True), 
                     StructField('Documentado', BooleanType(), True)])

* La siguiente celda creará una *dataframe* a partir de ```pandas_df``` con el *schema* correspondiente a ```schema```.

In [22]:
pandas_df.dtypes

Dirección      object
Rumbo          object
Pasajeros       int64
Documentado    object
dtype: object

In [23]:
df = spark.createDataFrame(data=pandas_df, schema=schema)

In [24]:
df.show()

+---------+--------+---------+-----------+
|Dirección|   Rumbo|Pasajeros|Documentado|
+---------+--------+---------+-----------+
|      Sur|    Este|       12|       true|
|    Norte|Noroeste|       24|       null|
|      Sur|   Norte|       32|       true|
|     Este|   Norte|        5|      false|
+---------+--------+---------+-----------+



In [25]:
df.printSchema()

root
 |-- Dirección: string (nullable = true)
 |-- Rumbo: string (nullable = true)
 |-- Pasajeros: byte (nullable = true)
 |-- Documentado: boolean (nullable = true)



In [26]:
df.schema

StructType([StructField('Dirección', StringType(), True), StructField('Rumbo', StringType(), True), StructField('Pasajeros', ByteType(), True), StructField('Documentado', BooleanType(), True)])

## Lectura y escritura de archivos para *dataframes*.

https://spark.apache.org/docs/latest/sql-data-sources-load-save-functions.html

### Lectura de datos hacia un *dataframe*.

En el caso de *PySpark*  es posible leer y crear dataframes a partir de distintos formatos de archivo que describan estructuras tabulares. La propiedad [```spark.read```](https://spark.apache.org/docs/3.1.2/api/python/reference/api/pyspark.sql.SparkSession.read.html) contiene una familia de métodos que pueden importar y convertir en *dataframes* diversos tipos de documento y de diversas fuentes.

```
df = spark.read.<método>(<ruta>)
```

### Escritura de datos desde un *dataframe*.

En el caso de *PySpark*  es posible leer y crear dataframes a partir de distintos formatos de archivo que describan estructuras tabulares. Las siguientes propiedades de los *dataframes* permiten exportar dataframes a distintos formatos y destinos.

* [```df.write```](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.write.html) perimte crear y escribir una serie de documentos en el fromato indicado.
* [```df.writeTo```](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.writeTo.html) permite añadir (*append*) datos a un documento o estructura existente.
* [```df.writeStream```](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.writeStream.html) permite enviar los datos del *dataframe* a un flujo.

```
df.write.<método>(<ruta>)
```
La propiedad ```df.write``` es una implementación del objeto [```DataFrameWriter```](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrameWriter.html#pyspark.sql.DataFrameWriter).


### Fuentes de datos compatibles para los *dataframes*.

https://spark.apache.org/docs/latest/sql-data-sources.html

 ### El método ```spark.read.parquet()```.
 
 [*Apache Parquet*](https://parquet.apache.org/) es un formato binario capaz de almacenar estructuras de datos y conservar sus *schemas*.
 
 El siguiente enlace apunta a la documentación de la lectura/escritura de documentos en formato *parquet* para *Spark*.
 
 https://spark.apache.org/docs/latest/sql-data-sources-parquet.html
 
 

**Ejemplo:**

* La siguiente celda importará los datos del archivo ```data/data_covid.parquet```.

In [28]:
df = spark.read.parquet('data/data_covid.parquet')

In [29]:
df

DataFrame[AGUASCALIENTES: bigint, BAJA CALIFORNIA: bigint, BAJA CALIFORNIA SUR: bigint, CAMPECHE: bigint, CHIAPAS: bigint, CHIHUAHUA: bigint, DISTRITO FEDERAL: bigint, COAHUILA: bigint, COLIMA: bigint, DURANGO: bigint, GUANAJUATO: bigint, GUERRERO: bigint, HIDALGO: bigint, JALISCO: bigint, MEXICO: bigint, MICHOACAN: bigint, MORELOS: bigint, NAYARIT: bigint, NUEVO LEON: bigint, OAXACA: bigint, PUEBLA: bigint, QUERETARO: bigint, QUINTANA ROO: bigint, SAN LUIS POTOSI: bigint, SINALOA: bigint, SONORA: bigint, TABASCO: bigint, TAMAULIPAS: bigint, TLAXCALA: bigint, VERACRUZ: bigint, YUCATAN: bigint, ZACATECAS: bigint, Nacional: bigint, index: timestamp]

In [30]:
df.show()

+--------------+---------------+-------------------+--------+-------+---------+----------------+--------+------+-------+----------+--------+-------+-------+------+---------+-------+-------+----------+------+------+---------+------------+---------------+-------+------+-------+----------+--------+--------+-------+---------+--------+-------------------+
|AGUASCALIENTES|BAJA CALIFORNIA|BAJA CALIFORNIA SUR|CAMPECHE|CHIAPAS|CHIHUAHUA|DISTRITO FEDERAL|COAHUILA|COLIMA|DURANGO|GUANAJUATO|GUERRERO|HIDALGO|JALISCO|MEXICO|MICHOACAN|MORELOS|NAYARIT|NUEVO LEON|OAXACA|PUEBLA|QUERETARO|QUINTANA ROO|SAN LUIS POTOSI|SINALOA|SONORA|TABASCO|TAMAULIPAS|TLAXCALA|VERACRUZ|YUCATAN|ZACATECAS|Nacional|              index|
+--------------+---------------+-------------------+--------+-------+---------+----------------+--------+------+-------+----------+--------+-------+-------+------+---------+-------+-------+----------+------+------+---------+------------+---------------+-------+------+-------+----------+-------

In [31]:
df.show(1)

+--------------+---------------+-------------------+--------+-------+---------+----------------+--------+------+-------+----------+--------+-------+-------+------+---------+-------+-------+----------+------+------+---------+------------+---------------+-------+------+-------+----------+--------+--------+-------+---------+--------+-------------------+
|AGUASCALIENTES|BAJA CALIFORNIA|BAJA CALIFORNIA SUR|CAMPECHE|CHIAPAS|CHIHUAHUA|DISTRITO FEDERAL|COAHUILA|COLIMA|DURANGO|GUANAJUATO|GUERRERO|HIDALGO|JALISCO|MEXICO|MICHOACAN|MORELOS|NAYARIT|NUEVO LEON|OAXACA|PUEBLA|QUERETARO|QUINTANA ROO|SAN LUIS POTOSI|SINALOA|SONORA|TABASCO|TAMAULIPAS|TLAXCALA|VERACRUZ|YUCATAN|ZACATECAS|Nacional|              index|
+--------------+---------------+-------------------+--------+-------+---------+----------------+--------+------+-------+----------+--------+-------+-------+------+---------+-------+-------+----------+------+------+---------+------------+---------------+-------+------+-------+----------+-------

### El método ```df.toPandas()```.

El método ```df.toPandas()``` permite crear un *dataframe* de *Pandas* a partir de un *dataframe* de *Apache Spark*.

https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.toPandas.html

* A continuación se usará el método ```df.toPandas()``` para mostrar el *dataframe* ```df``` de mejor manera.

In [32]:
df.toPandas()

/usr/local/spark/python/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,AGUASCALIENTES,BAJA CALIFORNIA,BAJA CALIFORNIA SUR,CAMPECHE,CHIAPAS,CHIHUAHUA,DISTRITO FEDERAL,COAHUILA,COLIMA,DURANGO,...,SINALOA,SONORA,TABASCO,TAMAULIPAS,TLAXCALA,VERACRUZ,YUCATAN,ZACATECAS,Nacional,index
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2020-02-26
1,0,0,0,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,4,2020-02-27
2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,2,2020-02-28
3,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,2020-02-29
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,2020-03-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
790,25,23,24,1,1,8,163,4,9,8,...,6,12,4,5,4,79,38,3,583,2022-04-26
791,21,25,18,3,1,9,200,3,6,1,...,10,7,5,8,2,51,25,5,556,2022-04-27
792,33,17,12,4,3,9,134,2,2,2,...,11,4,10,8,0,34,24,1,456,2022-04-28
793,19,15,6,1,1,2,63,2,2,0,...,1,2,3,3,0,39,27,3,255,2022-04-29


 ### El método ```spark.read.csv()```.
 
 Los documentos *CSV* *(comma separated values)* son documentos de texto en los que se ingresan datos usando un caracter como separador. Por lo general, este separador es el caracter ```,```. Los documento CSV no conservan el tipo de dato y todos los elmentos son cadenas de caracteres. *Spark* puede leer este tipo de formatos, pero es necesario indicarle el *schema* que utilizará.
 
 El siguiente enlace apunta a la documentación de la lectura/escritura de documentos en formato *CSV* para *Spark*.
 
 https://spark.apache.org/docs/latest/sql-data-sources-csv.html

**Ejemplos:**

* La siguiente celda leerá los datos dentro del archivo [```data/data_covid.csv```](data/data_covid.csv). En este caso sólo se indica la opción de que existe un encabezado en el documento.

In [33]:
df = spark.read.option("header","true").csv('data/data_covid.csv')
df.toPandas()

,index,AGUASCALIENTES,BAJA CALIFORNIA,BAJA CALIFORNIA SUR,CAMPECHE,CHIAPAS,CHIHUAHUA,DISTRITO FEDERAL,COAHUILA,COLIMA,...,SAN LUIS POTOSI,SINALOA,SONORA,TABASCO,TAMAULIPAS,TLAXCALA,VERACRUZ,YUCATAN,ZACATECAS,Nacional
0,2020-02-26,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2020-02-27,0,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,4
2,2020-02-28,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,2
3,2020-02-29,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
4,2020-03-01,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
790,2022-04-26,25,23,24,1,1,8,163,4,9,...,16,6,12,4,5,4,79,38,3,583
791,2022-04-27,21,25,18,3,1,9,200,3,6,...,21,10,7,5,8,2,51,25,5,556
792,2022-04-28,33,17,12,4,3,9,134,2,2,...,13,11,4,10,8,0,34,24,1,456
793,2022-04-29,19,15,6,1,1,2,63,2,2,...,7,1,2,3,3,0,39,27,3,255


In [34]:
df.printSchema()

root
 |-- index: string (nullable = true)
 |-- AGUASCALIENTES: string (nullable = true)
 |-- BAJA CALIFORNIA: string (nullable = true)
 |-- BAJA CALIFORNIA SUR: string (nullable = true)
 |-- CAMPECHE: string (nullable = true)
 |-- CHIAPAS: string (nullable = true)
 |-- CHIHUAHUA: string (nullable = true)
 |-- DISTRITO FEDERAL: string (nullable = true)
 |-- COAHUILA: string (nullable = true)
 |-- COLIMA: string (nullable = true)
 |-- DURANGO: string (nullable = true)
 |-- GUANAJUATO: string (nullable = true)
 |-- GUERRERO: string (nullable = true)
 |-- HIDALGO: string (nullable = true)
 |-- JALISCO: string (nullable = true)
 |-- MEXICO: string (nullable = true)
 |-- MICHOACAN: string (nullable = true)
 |-- MORELOS: string (nullable = true)
 |-- NAYARIT: string (nullable = true)
 |-- NUEVO LEON: string (nullable = true)
 |-- OAXACA: string (nullable = true)
 |-- PUEBLA: string (nullable = true)
 |-- QUERETARO: string (nullable = true)
 |-- QUINTANA ROO: string (nullable = true)
 |-- SAN 

* La siguiente celda leerá los datos dentro del archivo [```data/data_covid.csv```](data/data_covid.csv). En este caso se indican las opciones de que existe un encabezado en el documento y que se inferirá el *schema* de las columnnas.

In [35]:
df = spark.read.option("header","true").option("inferSchema", "true").csv('data/data_covid.csv')
df.toPandas()

/usr/local/spark/python/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,index,AGUASCALIENTES,BAJA CALIFORNIA,BAJA CALIFORNIA SUR,CAMPECHE,CHIAPAS,CHIHUAHUA,DISTRITO FEDERAL,COAHUILA,COLIMA,...,SAN LUIS POTOSI,SINALOA,SONORA,TABASCO,TAMAULIPAS,TLAXCALA,VERACRUZ,YUCATAN,ZACATECAS,Nacional
0,2020-02-26,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2020-02-27,0,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,4
2,2020-02-28,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,2
3,2020-02-29,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
4,2020-03-01,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
790,2022-04-26,25,23,24,1,1,8,163,4,9,...,16,6,12,4,5,4,79,38,3,583
791,2022-04-27,21,25,18,3,1,9,200,3,6,...,21,10,7,5,8,2,51,25,5,556
792,2022-04-28,33,17,12,4,3,9,134,2,2,...,13,11,4,10,8,0,34,24,1,456
793,2022-04-29,19,15,6,1,1,2,63,2,2,...,7,1,2,3,3,0,39,27,3,255


In [36]:
df.printSchema()

root
 |-- index: timestamp (nullable = true)
 |-- AGUASCALIENTES: integer (nullable = true)
 |-- BAJA CALIFORNIA: integer (nullable = true)
 |-- BAJA CALIFORNIA SUR: integer (nullable = true)
 |-- CAMPECHE: integer (nullable = true)
 |-- CHIAPAS: integer (nullable = true)
 |-- CHIHUAHUA: integer (nullable = true)
 |-- DISTRITO FEDERAL: integer (nullable = true)
 |-- COAHUILA: integer (nullable = true)
 |-- COLIMA: integer (nullable = true)
 |-- DURANGO: integer (nullable = true)
 |-- GUANAJUATO: integer (nullable = true)
 |-- GUERRERO: integer (nullable = true)
 |-- HIDALGO: integer (nullable = true)
 |-- JALISCO: integer (nullable = true)
 |-- MEXICO: integer (nullable = true)
 |-- MICHOACAN: integer (nullable = true)
 |-- MORELOS: integer (nullable = true)
 |-- NAYARIT: integer (nullable = true)
 |-- NUEVO LEON: integer (nullable = true)
 |-- OAXACA: integer (nullable = true)
 |-- PUEBLA: integer (nullable = true)
 |-- QUERETARO: integer (nullable = true)
 |-- QUINTANA ROO: integer (

* La siguiente celda definirá un *schema* para el *dataframe* resultante de extraer el archivo *CSV*.

In [37]:
schema = StructType([StructField('index', DateType(), True), StructField('AGUASCALIENTES', LongType(), True), StructField('BAJA CALIFORNIA', LongType(), True), StructField('BAJA CALIFORNIA SUR', LongType(), True), StructField('CAMPECHE', LongType(), True), StructField('CHIAPAS', LongType(), True), StructField('CHIHUAHUA', LongType(), True), StructField('DISTRITO FEDERAL', LongType(), True), StructField('COAHUILA', LongType(), True), StructField('COLIMA', LongType(), True), StructField('DURANGO', LongType(), True), StructField('GUANAJUATO', LongType(), True), StructField('GUERRERO', LongType(), True), StructField('HIDALGO', LongType(), True), StructField('JALISCO', LongType(), True), StructField('MEXICO', LongType(), True), StructField('MICHOACAN', LongType(), True), StructField('MORELOS', LongType(), True), StructField('NAYARIT', LongType(), True), StructField('NUEVO LEON', LongType(), True), StructField('OAXACA', LongType(), True), StructField('PUEBLA', LongType(), True), StructField('QUERETARO', LongType(), True), StructField('QUINTANA ROO', LongType(), True), StructField('SAN LUIS POTOSI', LongType(), True), StructField('SINALOA', LongType(), True), StructField('SONORA', LongType(), True), StructField('TABASCO', LongType(), True), StructField('TAMAULIPAS', LongType(), True), StructField('TLAXCALA', LongType(), True), StructField('VERACRUZ', LongType(), True), StructField('YUCATAN', LongType(), True), StructField('ZACATECAS', LongType(), True), StructField('Nacional', LongType(), True)])

In [38]:
df = spark.read.option("header","true").schema(schema).csv('data/data_covid.csv')
df.toPandas()

,index,AGUASCALIENTES,BAJA CALIFORNIA,BAJA CALIFORNIA SUR,CAMPECHE,CHIAPAS,CHIHUAHUA,DISTRITO FEDERAL,COAHUILA,COLIMA,...,SAN LUIS POTOSI,SINALOA,SONORA,TABASCO,TAMAULIPAS,TLAXCALA,VERACRUZ,YUCATAN,ZACATECAS,Nacional
0,2020-02-26,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2020-02-27,0,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,4
2,2020-02-28,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,2
3,2020-02-29,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
4,2020-03-01,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
790,2022-04-26,25,23,24,1,1,8,163,4,9,...,16,6,12,4,5,4,79,38,3,583
791,2022-04-27,21,25,18,3,1,9,200,3,6,...,21,10,7,5,8,2,51,25,5,556
792,2022-04-28,33,17,12,4,3,9,134,2,2,...,13,11,4,10,8,0,34,24,1,456
793,2022-04-29,19,15,6,1,1,2,63,2,2,...,7,1,2,3,3,0,39,27,3,255


In [39]:
df.printSchema()

root
 |-- index: date (nullable = true)
 |-- AGUASCALIENTES: long (nullable = true)
 |-- BAJA CALIFORNIA: long (nullable = true)
 |-- BAJA CALIFORNIA SUR: long (nullable = true)
 |-- CAMPECHE: long (nullable = true)
 |-- CHIAPAS: long (nullable = true)
 |-- CHIHUAHUA: long (nullable = true)
 |-- DISTRITO FEDERAL: long (nullable = true)
 |-- COAHUILA: long (nullable = true)
 |-- COLIMA: long (nullable = true)
 |-- DURANGO: long (nullable = true)
 |-- GUANAJUATO: long (nullable = true)
 |-- GUERRERO: long (nullable = true)
 |-- HIDALGO: long (nullable = true)
 |-- JALISCO: long (nullable = true)
 |-- MEXICO: long (nullable = true)
 |-- MICHOACAN: long (nullable = true)
 |-- MORELOS: long (nullable = true)
 |-- NAYARIT: long (nullable = true)
 |-- NUEVO LEON: long (nullable = true)
 |-- OAXACA: long (nullable = true)
 |-- PUEBLA: long (nullable = true)
 |-- QUERETARO: long (nullable = true)
 |-- QUINTANA ROO: long (nullable = true)
 |-- SAN LUIS POTOSI: long (nullable = true)
 |-- SINALOA

* Las siguientes celda escribirán los datos del *dataframe* en diversos formatos.

In [40]:
df.write.csv('datos')

In [41]:
df.write.parquet('datos_parquet')

In [42]:
df.write.json('datos_json')

In [43]:
rm -rf datos_json

In [44]:
df.write.option('encoding', 'UTF-8').json('datos_json')

In [45]:
df_nuevo = spark.read.option('encoding', 'UTF-8').json('datos_json')

In [46]:
df.toPandas()

,index,AGUASCALIENTES,BAJA CALIFORNIA,BAJA CALIFORNIA SUR,CAMPECHE,CHIAPAS,CHIHUAHUA,DISTRITO FEDERAL,COAHUILA,COLIMA,...,SAN LUIS POTOSI,SINALOA,SONORA,TABASCO,TAMAULIPAS,TLAXCALA,VERACRUZ,YUCATAN,ZACATECAS,Nacional
0,2020-02-26,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2020-02-27,0,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,4
2,2020-02-28,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,2
3,2020-02-29,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
4,2020-03-01,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
790,2022-04-26,25,23,24,1,1,8,163,4,9,...,16,6,12,4,5,4,79,38,3,583
791,2022-04-27,21,25,18,3,1,9,200,3,6,...,21,10,7,5,8,2,51,25,5,556
792,2022-04-28,33,17,12,4,3,9,134,2,2,...,13,11,4,10,8,0,34,24,1,456
793,2022-04-29,19,15,6,1,1,2,63,2,2,...,7,1,2,3,3,0,39,27,3,255


## Ejemplo de funciones de los *dataframes*.

https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.sql.DataFrame.html

In [47]:
df.select('index','AGUASCALIENTES').where(df.AGUASCALIENTES >1000).show()

+----------+--------------+
|     index|AGUASCALIENTES|
+----------+--------------+
|2022-01-17|          1505|
|2022-01-18|          1169|
|2022-01-20|          1103|
+----------+--------------+



In [48]:
from datetime import date

In [49]:
df.select('index','AGUASCALIENTES').where(df.index == date(2020,12,20)).show()

+----------+--------------+
|     index|AGUASCALIENTES|
+----------+--------------+
|2020-12-20|            29|
+----------+--------------+



In [50]:
df.select('index','AGUASCALIENTES').filter(df.index >= date(2020,12,1)).show()

+----------+--------------+
|     index|AGUASCALIENTES|
+----------+--------------+
|2020-12-01|           203|
|2020-12-02|           158|
|2020-12-03|           137|
|2020-12-04|           115|
|2020-12-05|            40|
|2020-12-06|            47|
|2020-12-07|           148|
|2020-12-08|           136|
|2020-12-09|           114|
|2020-12-10|           101|
|2020-12-11|            95|
|2020-12-12|            26|
|2020-12-13|            27|
|2020-12-14|           118|
|2020-12-15|           117|
|2020-12-16|            94|
|2020-12-17|           105|
|2020-12-18|           100|
|2020-12-19|            32|
|2020-12-20|            29|
+----------+--------------+
only showing top 20 rows



In [51]:
df.select('index').count()

795

In [52]:
df.select('index', 'AGUASCALIENTES').sort('AGUASCALIENTES').toPandas()

,index,AGUASCALIENTES
0,2020-02-26,0
1,2020-02-27,0
2,2020-02-28,0
3,2020-02-29,0
4,2020-03-01,0
...,...,...
790,2022-01-24,944
791,2022-01-25,953
792,2022-01-20,1103
793,2022-01-18,1169


In [53]:
df.select('index', 'AGUASCALIENTES').sort('AGUASCALIENTES', ascending=False).limit(12).toPandas()

,index,AGUASCALIENTES
0,2022-01-17,1505
1,2022-01-18,1169
2,2022-01-20,1103
3,2022-01-25,953
4,2022-01-24,944
5,2022-01-27,885
6,2022-01-19,863
7,2022-01-11,860
8,2022-01-14,855
9,2022-01-31,817


In [54]:
df.select('index', 'AGUASCALIENTES', 'Nacional').sort('AGUASCALIENTES', ascending=False).limit(10).toPandas()

,index,AGUASCALIENTES,Nacional
0,2022-01-17,1505,79651
1,2022-01-18,1169,74129
2,2022-01-20,1103,64525
3,2022-01-25,953,50093
4,2022-01-24,944,60752
5,2022-01-27,885,40582
6,2022-01-19,863,69731
7,2022-01-11,860,70936
8,2022-01-14,855,72308
9,2022-01-31,817,36673


In [55]:
df.select('index', 'AGUASCALIENTES', 'Nacional').sort('Nacional', ascending=False).limit(10).toPandas()

,index,AGUASCALIENTES,Nacional
0,2022-01-17,1505,79651
1,2022-01-12,785,74420
2,2022-01-18,1169,74129
3,2022-01-13,708,73216
4,2022-01-14,855,72308
5,2022-01-11,860,70936
6,2022-01-19,863,69731
7,2022-01-10,685,68043
8,2022-01-20,1103,64525
9,2022-01-24,944,60752


In [56]:
dias_ags = df.select('index', 'AGUASCALIENTES').sort('AGUASCALIENTES', ascending=False).limit(10)
dias_nacional = df.select('index', 'Nacional').sort('Nacional', ascending=False).limit(10)

In [57]:
dias_ags.join(dias_nacional, 'index').show()

+----------+--------------+--------+
|     index|AGUASCALIENTES|Nacional|
+----------+--------------+--------+
|2022-01-17|          1505|   79651|
|2022-01-18|          1169|   74129|
|2022-01-20|          1103|   64525|
|2022-01-24|           944|   60752|
|2022-01-19|           863|   69731|
|2022-01-11|           860|   70936|
|2022-01-14|           855|   72308|
+----------+--------------+--------+



In [58]:
dias_ags.join(dias_nacional, 'index').count()

7

In [59]:
dias_ags.join(dias_nacional, 'index', how='full').show()

+----------+--------------+--------+
|     index|AGUASCALIENTES|Nacional|
+----------+--------------+--------+
|2022-01-10|          null|   68043|
|2022-01-11|           860|   70936|
|2022-01-12|          null|   74420|
|2022-01-13|          null|   73216|
|2022-01-14|           855|   72308|
|2022-01-17|          1505|   79651|
|2022-01-18|          1169|   74129|
|2022-01-19|           863|   69731|
|2022-01-20|          1103|   64525|
|2022-01-24|           944|   60752|
|2022-01-25|           953|    null|
|2022-01-27|           885|    null|
|2022-01-31|           817|    null|
+----------+--------------+--------+



In [60]:
dias_ags.join(dias_nacional, 'index').select('index').show()

+----------+
|     index|
+----------+
|2022-01-17|
|2022-01-18|
|2022-01-20|
|2022-01-24|
|2022-01-19|
|2022-01-11|
|2022-01-14|
+----------+



In [ ]:
spark.stop()

<p style="text-align: center"><a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Licencia Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/80x15.png" /></a><br />Esta obra está bajo una <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Licencia Creative Commons Atribución 4.0 Internacional</a>.</p>
<p style="text-align: center">&copy; José Luis Chiquete Valdivieso. 2023.</p>